# Dataset, documents, FAISS; retriever

## 🔹 Load the dataset containing the tuples `(query, correct_answer, distractor_1, distractor_2)` and the one containing the documents

In [1]:
import torch

torch.cuda.empty_cache()

In [1]:
from datasets import load_dataset

dataset = load_dataset('saracandu/hotpotQA_nli', split="train", trust_remote_code=True)
dataset

Dataset({
    features: ['question', 'answer', 'options', 'first nli', 'second nli', 'type', 'level', 'passages', 'BART1', 'BART2', 'ROBERTA1', 'ROBERTA2'],
    num_rows: 352
})

In [2]:
dataset[10]

{'question': 'Who was inducted into the Rock and Roll Hall of Fame, David Lee Roth or Cia Berg?',
 'answer': 'David Lee Roth',
 'options': "['Cia Berg', 'David Lee Roth']",
 'first nli': 'David Lee Roth was inducted into the Rock and Roll Hall of Fame before Cia Berg',
 'second nli': 'Cia Berg was inducted into the Rock and Roll Hall of Fame before David Lee Roth',
 'type': 'comparison',
 'level': 'medium',
 'passages': 'Cia Berg (born 2 December 1963), now known as Cia Soro, is a Swedish television presenter and singer. She was at one time the lead singer of the Swedish rock band Whale, who released the single "Hobo Humpin\' Slobo Babe". David Lee Roth (born October 10, 1954) is an American rock vocalist, musician, songwriter, actor, author, and former radio personality. In 2007, he was inducted into the Rock and Roll Hall of Fame.',
 'BART1': 0.885607123374939,
 'BART2': 0.0146357780322432,
 'ROBERTA1': 0.3287698328495025,
 'ROBERTA2': 0.0123543506488204}

# Model loading and dataset selection (for testing purposes)

## ▪️ Upload the model: 

In [ ]:
# do not run this unless necessary!

from huggingface_hub import login
login()

In [11]:
import pandas as pd
import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import warnings
warnings.filterwarnings("ignore")

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", use_fast=False)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
from guidance import models, select

new_model = models.Transformers(model, tokenizer, temperature=0.0)

# new_model + f'Do you want a joke or a poem? A ' + select(['joke', 'poem'])

## ▪️ Test with `guidance`: 

In [8]:
system_message = """You are a multiple-choice question answering assistant.
Choose which of the following options: a star, a planet, a galaxy is the object below.

Object: the sun
"""

new_model + system_message + select(["a planet", "a galaxy", "a star"])

In [9]:
from langchain import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
You have a suggestion on which answer is the most appropriate, that is treated as context. Use the suggestion to choose the most proper option.
You also have an attempt of answer that you are suggested to neglect. 

Question: {question}
Attempt: {candidate_answer}
Context: {critique}

The most proper option between {options} is:
"""
)

augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "candidate_answer": itemgetter("candidate_answer"),
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }

synthesis_chain = augmentation | prompt_template 

In [10]:
# shuffles the order of the vector containing the correct answer and the two distractors
# returns another vector, shuffled
import re
import random

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

In [11]:
def synthesisGeneration(query, prompt_template, merged, candidate_answer, critique, sources):
    # merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'candidate_answer': candidate_answer,
                                            'critique': critique,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    new_model + normal_string + select(merged)

In [12]:
synthesisGeneration('what is the sun', prompt_template, ['star', 'planet'], 'planet', 
                    'the correct answer is: a star since bot an asteroid and a planet are inadequate and not supported by the context', 
                    'The Sun is the star at the center of the Solar System. It is a massive, nearly perfect sphere of hot plasma, heated to incandescence by nuclear fusion reactions in its core, radiating the energy from its surface mainly as visible light and infrared radiation with 10% at ultraviolet energies.')

In [13]:
source = """ Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. 
Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. 
In May 1846 it was merged into "Godey\'s Lady\'s Book. First for Women is a woman\'s magazine published by Bauer Media Group in the USA.  
The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies.
    """

synthesisGeneration('Which magazine was started first Arthur\'s Magazine or First for Women?', 
                    prompt_template, ['First for Women', 'Arthur\'s Magazine'], 'First for Women', 
                    'the correct answer is Arthur\'s Magazine and the context agrees', 
                    source)

In [14]:
source = """ "The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group. 
The Oberoi Group is a hotel company with its head office in Delhi. 
Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.".
    """

synthesisGeneration('The Oberoi family is part of a hotel company that has a head office in what city?', 
                    prompt_template, ['Delhi', 'Sammardenchia'], 'Sammardenchia', 
                    'Delhi is correct', 
                    source)

In [15]:
source = """ Allison Beth Allie Goertz (born March 2, 1991) is an American musician. 
Goertz is known for her satirical songs based on various pop culture topics. Her videos are posted on YouTube under the name of Cossbysweater. 
Subjects of her songs have included the film The Room, the character Milhouse from the television show The Simpsons, and the game Dungeons & Dragons. 
Her style has been compared to that of Bo Burnham. In December 2015, Goertz released a concept album based on the Adult Swim series Rick and Morty, Sad Dance Songs, 
with the album\'s cover emulating the animation and logo of the series.  The album was made possible through Kickstarter. 
She is co-host of Everything's Coming Up Podcast, a Simpsons-focused podcast along with Julia Prescott. 
Milhouse Mussolini van Houten is a fictional character featured in the animated television series The Simpsons, voiced by Pamela Hayden, and created by Matt Groening 
who named the character after President Richard Nixon\'s middle name. Later in the series, it is revealed that Milhouse\'s middle name is Mussolini. "
    """

synthesisGeneration('Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?', 
                    prompt_template, ['Nixon', 'Obama'], 'Obama', 
                    'the correct answer is Nixon', 
                    source)

In [16]:
source = """ 
Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 – June 16, 1979) was an American film director best known for the movie Rebel Without a Cause. 
Elia Kazan (born Elias Kazantzoglou September 7, 1909 – September 28, 2003) was a Greek-American director, producer, writer and actor, described by The New York Times as one of the most honored and influential directors in Broadway and Hollywood history.
    """

synthesisGeneration('What profession does Nicholas Ray and Elia Kazan have in common?', 
                    prompt_template, ['director', 'writer'], 'director', 
                    'the correct answer is writer', 
                    source)

## ▪️ Select a subset of the true dataset as a test

In [2]:
# shuffles the order of the vector containing the correct answer and the two distractors
# returns another vector, shuffled
import re
import random

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

In [3]:
# select a subset of the queries, just for test:
first_queries = dataset['question']

# same for correct answers and distractors:
correct_answers = dataset['answer']
possibilities = dataset['options']

# and for the sources:
sources = dataset['passages']

#nli
first_nli = dataset['first nli']
second_nli = dataset['second nli']

bart1 = dataset['BART1']
bart2 = dataset['BART2']

rob1 = dataset['ROBERTA1']
rob2 = dataset['ROBERTA2']

In [4]:
N_examples = len(first_queries)

# Thesis

## 🔹 PromptTemplate definition and a LLMChain for the **thesis** 

In [12]:
def create_message(question, options, context):
    options_str = '", "'.join(options)
    content = f"""

    Now do the same for this question: "{question}", where options: ["{options_str}"]. Assistant:
    """

    user_content = "Answer to the following question: " + question + " providing one of these options as answer: " + str(options) + "Assistant:"

    messages = [
        {"role": "system", "content": """
        You are an helpful AI assistant. You have to provide helpful answers to the user’s questions based on the context: 
        """ + context},
        {"role": "user", "content": user_content}
    ]

    return messages

In [13]:
create_message(first_queries[0], possibilities[0], sources[0])

[{'role': 'system',
  'content': '\n        You are an helpful AI assistant. You have to provide helpful answers to the user’s questions based on the context: \n        Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book". First for Women is a woman\'s magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies.'},
 {'role': 'user',
  'content': 'Answer to the following question: Which magazine was started first, Arthur\'s Magazine or First for Women? providing one of these options as answer: ["Arthur\'s Magazine", \'First for Women\']Assistant:'}]

## 🔹 Function that generates the output given the prompt, the question and the set of options

In [14]:
import ast

def thesisGeneration(query, merged, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = create_message(query, merged, sources)
    # normal_string = clean_text(augmented_prompt)
    ans = new_model + str(augmented_prompt) + select([clean_text(merged[0]), clean_text(merged[1])])
    return str(ans)

In [15]:
def extract_answer(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("}]")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("}]")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

## 🔹 Test: how well the thesis alone is able to perform?

In [16]:
answers = []
for i in range(N_examples):
    answers.append(extract_answer(thesisGeneration(first_queries[i], possibilities[i], sources[i])))

In [17]:
answers[:10]

['First for Women',
 'Henri Leconte',
 'The Wolfhounds',
 'no',
 'yes',
 'San Francisco',
 'Aleksander Ford',
 'yes',
 'director',
 'The Saimaa Gesture']

In [25]:
df = {
    'query': first_queries,
    'correct': correct_answers,
    'thesis': answers,
    'context': sources
}

df = pd.DataFrame(df)
df.to_csv('phi-mini-thesis.csv')

# Antithesis

In [5]:
bart_answers = []

for i in range(len(first_queries)):

    if bart1[i] > bart2[i]:
        bart_answers.append([first_nli[i], bart1[i]])
    else:
        bart_answers.append([second_nli[i], bart2[i]])

In [6]:
rob_answers = []

for i in range(len(first_queries)):

    if rob1[i] > rob2[i]:
        rob_answers.append([first_nli[i], rob1[i]])
    else:
        rob_answers.append([second_nli[i], rob2[i]])

In [8]:
import pandas as pd

thesis_df = pd.read_csv('phi-mini-thesis.csv')

query = thesis_df['query']
correct = thesis_df['correct']
thesis = thesis_df['thesis']
sources = thesis_df['context']

# Pre Synthesis

In [9]:
# prompt per baseline-2
system_message = """
    You are an helpful AI assistant.
    You are asked to determine the most correct answer for a given question.
    You have at disposal a first tentative answer (a candidate answer) and another opinion on which should be the correct option according to context (a suggestion).
    
    They could agree on the correct option; in this case, directly output the option on which they agree.
    If instead they disagree, use the context to determine the correct answer for the question, given the set of possible options.
    
    The goal of the assistant is to decree which is the most correct answer to the question between the available options. 
    Answer by explicitly reporting the correct answer to you.
"""


user_message = """
    Question: {question}
    Options: {options}
    Candidate answer: {candidate_answer}
    Suggestion: {critique}
    Which of the candidate answers {options} is the most proper answer for the question?

"""

In [14]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.5,
    return_full_text=True,
    max_new_tokens=400,
    top_p=0.0
)

def query_model(
        system_message,
        user_message,
        temperature = 0.0,
        max_length=400
        ):

    user_message = "Question: " + user_message + " Correct answer:"
    messages = [
        {"role": "System", "content": system_message},
        {"role": "User", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        top_p=0.0,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )

    answer = sequences[0]['generated_text']
    return answer 

In [22]:
def preSynthGeneration(query, merged, candidate_answer, critique, sources):
    merged = ast.literal_eval(merged)
    second_answer = query_model(system_message.format(context = sources),
    user_message.format(question=query, options = merged, candidate_answer = candidate_answer, critique=critique, context = sources,), max_length=1024)
    return second_answer

In [23]:
pre_answers = []
for i in range(len(first_queries)):
    pre_answers.append(preSynthGeneration(first_queries[i], possibilities[i], answers[i], bart_answers[i], sources[i]))

KeyboardInterrupt: 

In [ ]:
pre_answers[:10]

# Synthesis

## 🔺 PromptTemplate definition and a LLMChain for the **synthesis** 

In [ ]:
from langchain import PromptTemplate
from operator import itemgetter

prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
Choose the most proper option between {options} that best matches with the suggestion. 

Question: {question}
Context: {critique}
Sources: {context}

Assistant:
"""
)

augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }

synthesis_chain = augmentation | prompt_template 

## 🔺 Function to generate the synthesis given literally everything

In [ ]:
def synthesisGeneration(query, prompt_template, merged, pre_answer, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'critique': pre_answer,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select([clean_text(merged[0]), clean_text(merged[1])])
    return str(ans)

In [ ]:
def extract_answer(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("assistant:\n")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("assistant:\n")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [ ]:
syn_answers = []
for i in range(len(first_queries)):
    syn_answers.append(extract_answer(
        synthesisGeneration(
            first_queries[i], prompt_template, possibilities[i], 
            bart_answers[i], sources[i])))

In [ ]:
syn_answers[:10]

# Dataset conversion and performances

In [ ]:
df = {
    'query': first_queries,
    'correct': correct_answers,
    'thesis': answers,
    'antithesis': ant_answers,
    'pre-synthesis': pre_answers,
    'synthesis': syn_answers,
    'context': sources
} 

In [ ]:
import pandas as pd

df = pd.DataFrame(df)
df.head()

In [ ]:
# df.to_csv('baseline-true.csv')

In [ ]:
# Funzione per rimuovere le quadre e ottenere solo il contenuto
def remove_brackets(s):
    return s.strip("[] ")

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    text = re.sub(r'[^\w\s.,!?\'"\-:;()]+', '', text)  # Rimuove i caratteri speciali
    text = re.sub(r"['\"-]", '', text)  # Rimuove apostrofi, virgolette e trattini
    text = text.lower()  # Converte in minuscolo
    return text

# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text)
df['thesis'] = df['thesis'].apply(clean_text)
df['synthesis'] = df['synthesis'].apply(clean_text)

df.head()

In [ ]:
# Conta quante righe combaciano e quante no
matches = 0
non_matches = 0
which_ones = []

for index, row in df.iterrows():
    correct_answer = str(row['correct']).strip()
    thesis = str(row['thesis']).strip()
    
    if correct_answer == thesis:
        matches += 1
    else:
        non_matches += 1
        which_ones.append("thesis: {}, Correct: {}".format(thesis, correct_answer))

print(f"Number of matches: {matches}")
print(f"Number of non-matches: {non_matches}")

In [ ]:
# Conta quante righe combaciano e quante no
matches = 0
non_matches = 0
which_ones_syn = []

for index, row in df.iterrows():
    correct_answer = str(row['correct']).strip()
    thesis = str(row['synthesis']).strip()
    
    if correct_answer == thesis:
        matches += 1
    else:
        # print("Synthesis: {}, Correct: {}".format(thesis, correct_answer))
        non_matches += 1
        which_ones_syn.append("Synthesis: {}, Correct: {}".format(thesis, correct_answer))

print(f"Number of matches: {matches}")
print(f"Number of non-matches: {non_matches}")

In [ ]:
df['goat'] = syn_answers
df['goat'] = df['goat'].apply(clean_text)
df.head()

In [47]:
# Conta quante righe combaciano e quante no
matches = 0
non_matches = 0

for index, row in df.iterrows():
    correct_answer = str(row['correct']).strip()
    thesis = str(row['goat']).strip()
    
    if correct_answer == thesis:
        matches += 1
    else:
        # print("Synthesis: {}, Correct: {}".format(thesis, correct_answer))
        non_matches += 1
        which_ones_syn.append("GOAT: {}, Correct: {}".format(thesis, correct_answer))

print(f"Number of matches: {matches}")
print(f"Number of non-matches: {non_matches}")

Number of matches: 314
Number of non-matches: 38


In [48]:
df.to_csv('phi-mini-baseline.csv')